# **Практика**

Наша практика будет основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа).

[**DATA**](https://lms.skillfactory.ru/assets/courseware/v1/9f2add5bca59f8c4df927432d605fff3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/_train_sem09__1_.zip)

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец **Activity** содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1];  
Остальные столбцы **D1-D1776** представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

![](https://lms.skillfactory.ru/assets/courseware/v1/243fdc2d8abf9f176c01a429f1ae68ab/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-3-ml-7-10.png)

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать **F1-score**.

1. Необходимо обучить две модели: логистическую регрессию и случайный лес.
2. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. 
   * Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

***
## **Библиотеки и "сырые" модели**

In [34]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
import hyperopt
import optuna

from sklearn import linear_model, tree, ensemble, metrics, preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import hp, fmin, tpe, Trials

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
plt.style.use('seaborn')
random_state = 42
data = pd.read_csv('data/train_sem.csv')

X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = random_state, test_size = 0.2)


In [35]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state = random_state, max_iter=50)
#Обучаем модель
log_reg.fit(X_train, y_train)
# Выводим значения метрики 
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78


In [36]:
# Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=random_state, n_estimators=50)
# Обучаем модель
rf.fit(X_train, y_train)
# Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 1.00
Test: 0.79


***
## **GRIDSEARCHCV**

### **LogisticRegression**

In [37]:
# Указываем сетку гиперпарметров, создаем объект гридсёрч
# и выполняем поиск лучших для метрики ф1 гиперпараметров
param_grid = [
              {'penalty': ['l2', 'none'],
              'solver': ['lbfgs', 'sag'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]},
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search = GridSearchCV(
    estimator=log_reg, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)

# %time - замеряет время выполнения
%time grid_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 3.52 s
Wall time: 46.2 s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


### **RandomForest**

In [38]:
# Указываем сетку гиперпарметров, создаем объект гридсёрч
# и выполняем поиск лучших для метрики ф1 гиперпараметров
param_grid = {'n_estimators': list(range(80, 200, 50)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }

grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)

# %time - замеряет время выполнения
%time grid_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 2.12 s
Wall time: 14.8 s
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 180}


***
##  **RANDOMIZEDSEARCHCV**

### **LogisticRegression**

In [39]:
# Указываем подбираемые комбинации гиперпараметров, создаем объект рандомсёрч
# и выполняем поиск лучших для метрики ф1 гиперпараметров

param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},

random_search = RandomizedSearchCV(
    estimator=log_reg, 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50, 
    n_jobs = -1
)

%time random_search.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 2.55 s
Wall time: 29.7 s
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.01}


### **RandomForest**

In [40]:
# Указываем подбираемые комбинации гиперпараметров, создаем объект рандомсёрч
# и выполняем поиск лучших для метрики ф1 гиперпараметров

param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 2.39 s
Wall time: 33.7 s
f1_score на обучающем наборе: 0.95
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 5, 'max_depth': 22}


***
## **HYPEROPT**

### **RandomForest**

In [41]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    # обучаем модель
    model.fit(X, y)
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [42]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [02:41<00:00,  3.22s/trial, best loss: -0.8201982028293425]
Наилучшие значения гиперпараметров {'max_depth': 15.0, 'min_samples_leaf': 2.0, 'n_estimators': 142.0}
CPU times: total: 1min 16s
Wall time: 2min 41s


In [43]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.98
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.81


### **LogisticRegression**

In [44]:
# списки вариантов гиперпараметров
penalty = ['none', 'l2']
solver = ['lbfgs', 'sag']
c_parameter = [0.01,0.02,0.03,0.04,0.05,0.07,0.09,1]
# пространство поиска гиперпараметров
space={'penalty': hp.choice(label='penalty', 
                          options=penalty),
       'solver' : hp.choice(label='solver', 
                          options=solver),
       'C': hp.choice(label='C', 
                          options=c_parameter)
      }

def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': params['solver'], 
             'C': params['C']
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state = random_state)
    # обучаем модель
    model.fit(X, y)
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [45]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [04:22<00:00,  5.25s/trial, best loss: -0.7949189441592461]
Наилучшие значения гиперпараметров {'C': 1, 'penalty': 1, 'solver': 1}
CPU times: total: 2min 29s
Wall time: 4min 22s


In [46]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=random_state, 
    penalty=penalty[best['penalty']],
    solver=solver[best['solver']],
    C=c_parameter[best['C']]
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.83
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78


***
## **OPTUNA**

### **RandomForest**

In [47]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [48]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-10-21 19:23:15,057] A new study created in memory with name: RandomForestClassifier
[I 2022-10-21 19:23:18,856] Trial 0 finished with value: 0.8096124499917643 and parameters: {'n_estimators': 141, 'max_depth': 17, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8096124499917643.
[I 2022-10-21 19:23:24,003] Trial 1 finished with value: 0.8127014683377096 and parameters: {'n_estimators': 196, 'max_depth': 27, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.8127014683377096.
[I 2022-10-21 19:23:26,848] Trial 2 finished with value: 0.799008415174908 and parameters: {'n_estimators': 127, 'max_depth': 27, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.8127014683377096.
[I 2022-10-21 19:23:29,336] Trial 3 finished with value: 0.8003164546167211 and parameters: {'n_estimators': 104, 'max_depth': 30, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.8127014683377096.
[I 2022-10-21 19:23:33,228] Trial 4 finished with value: 0.7994203241452389 and parameters: {'n_e

CPU times: total: 31.8 s
Wall time: 1min 14s


In [49]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80


### **LogisticRegression**

In [50]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['none', 'l2'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
  C = trial.suggest_categorical('C', [0.01,0.02,0.03,0.04,0.05,0.07,0.09,1])

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [51]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-10-21 19:24:31,651] A new study created in memory with name: LogisticRegression
[I 2022-10-21 19:24:33,313] Trial 0 finished with value: 0.7897018785888601 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.02}. Best is trial 0 with value: 0.7897018785888601.
[I 2022-10-21 19:24:35,208] Trial 1 finished with value: 0.7583522396374635 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.04}. Best is trial 0 with value: 0.7897018785888601.
[I 2022-10-21 19:24:37,083] Trial 2 finished with value: 0.7583522396374635 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.04}. Best is trial 0 with value: 0.7897018785888601.
[I 2022-10-21 19:24:38,977] Trial 3 finished with value: 0.7583522396374635 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.05}. Best is trial 0 with value: 0.7897018785888601.
[I 2022-10-21 19:24:40,970] Trial 4 finished with value: 0.7583522396374635 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.02}. Best is

CPU times: total: 57.5 s
Wall time: 1min 38s


In [52]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
